## Practica 1 - Computer Vision

# Importación de librerías

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os

# Exploración de archivos

In [2]:
# Rutas a los archivos de anotaciones
path_train = os.path.join(os.getcwd(), 'data', 'practica_1_dataset', 'train', 'annotations.csv')
path_valid = os.path.join(os.getcwd(), 'data', 'practica_1_dataset', 'valid', 'annotations.csv')
path_test = os.path.join(os.getcwd(), 'data', 'practica_1_dataset', 'test', 'annotations.csv')

# Cargar los DataFrames
df_train = pd.read_csv(path_train)
df_valid = pd.read_csv(path_valid)
df_test = pd.read_csv(path_test)

# Contar instancias por clase en cada DataFrame
train_class_counts = df_train['class'].value_counts()
valid_class_counts = df_valid['class'].value_counts()
test_class_counts = df_test['class'].value_counts()

# Combinar los DataFrames para contar las instancias generales por clase
df_combined = pd.concat([df_train, df_valid, df_test])
general_class_counts = df_combined['class'].value_counts()

# Obtener los valores únicos de la columna 'class' del conjunto combinado
valores_unicos = df_combined['class'].unique().tolist()

# Mostrar los resultados
print("Train class counts:")
print(train_class_counts)
print("\nValid class counts:")
print(valid_class_counts)
print("\nTest class counts:")
print(test_class_counts)
print("\nGeneral class counts:")
print(general_class_counts)
print("\nValores únicos en el dataset combinado:")
print(valores_unicos)

Train class counts:
class
fish         1961
jellyfish     385
penguin       330
shark         259
puffin        175
stingray      136
starfish       78
Name: count, dtype: int64

Valid class counts:
class
fish         459
jellyfish    155
penguin      104
puffin        74
shark         57
stingray      33
starfish      27
Name: count, dtype: int64

Test class counts:
class
fish         249
jellyfish    154
penguin       82
shark         38
puffin        35
stingray      15
starfish      11
Name: count, dtype: int64

General class counts:
class
fish         2669
jellyfish     694
penguin       516
shark         354
puffin        284
stingray      184
starfish      116
Name: count, dtype: int64

Valores únicos en el dataset combinado:
['starfish', 'shark', 'fish', 'puffin', 'stingray', 'penguin', 'jellyfish']


In [3]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import shutil

# Directorios de datos originales
base_dir = "data/practica_1_dataset"
train_dir = os.path.join(base_dir, "train")
valid_dir = os.path.join(base_dir, "valid")

# Directorios para datos procesados
processed_base = "data/processed"
processed_train_dir = os.path.join(processed_base, "train")
processed_valid_dir = os.path.join(processed_base, "validation")

os.makedirs(processed_train_dir, exist_ok=True)
os.makedirs(processed_valid_dir, exist_ok=True)

# Mapeo de clases a identificadores
class_mapping = {
  "fish": 1,
  "jellyfish": 2,
  "penguin": 3,
  "shark": 4,
  "puffin": 5,
  "stingray": 6,
  "starfish": 7
}

def process_folder(input_folder, output_folder, ann_df):
  """
  Procesa las imágenes en 'input_folder' y guarda en 'output_folder':
    - Copia la imagen original.
    - Si la imagen contiene más de una clase, genera una imagen sintética donde se reemplazan
      los píxeles de la clase predominante por el color medio del resto de la imagen.
    - Se registra la información en new_dataset.csv:
        * Para la imagen original se utiliza la primera clase (más frecuente).
        * Para la imagen sintética se utiliza la segunda clase (si existe).
  """
  info = []  # Lista para almacenar la información que se registrará en el CSV

  # Se obtienen los archivos de imagen (.jpg, .jpeg, .png)
  image_files = [f for f in os.listdir(input_folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

  for name in tqdm(image_files, desc=f"Procesando {os.path.basename(input_folder)}"):
    path = os.path.join(input_folder, name)
    image = Image.open(path).convert("RGB")
    w, h = image.size
    mask = np.zeros((h, w), dtype=np.uint8)
    
    # Asignar en la máscara el identificador de cada clase según las anotaciones
    for _, row in ann_df[ann_df["filename"] == name].iterrows():
      cid = class_mapping[row["class"]]
      mask[int(row["ymin"]):int(row["ymax"]), int(row["xmin"]):int(row["xmax"])] = cid

    # Se obtienen las clases presentes y se elimina el fondo (0)
    uniques, counts = np.unique(mask, return_counts=True)
    counts_dict = dict(zip(uniques, counts))
    counts_dict.pop(0, None)  # Se elimina el fondo

    # Se ordenan las clases por cantidad de píxeles (de mayor a menor)
    sorted_classes = sorted(counts_dict.items(), key=lambda x: x[1], reverse=True)

    # Verificar que sorted_classes no esté vacío antes de acceder a su primer elemento
    if not sorted_classes:
      print(f"No se encontraron anotaciones para la imagen: {name}")
      continue  # Salta al siguiente elemento

    prim_id, prim_area = sorted_classes[0]
    prim_label = next(label for label, idx in class_mapping.items() if idx == prim_id)

    # Registro para la imagen original
    info.append({"filename": name, "predominant_class": prim_label, "area": prim_area})
    shutil.copy(path, os.path.join(output_folder, name))
    
    # Solo se genera la imagen sintética si hay más de una clase anotada
    if len(sorted_classes) > 1:
      sec_id, sec_area = sorted_classes[1]
      sec_label = next(label for label, idx in class_mapping.items() if idx == sec_id)

      arr = np.array(image)
      mask_prim = (mask == prim_id)
      mean_color = arr[~mask_prim].mean(axis=0).astype(np.uint8)
      syn_arr = arr.copy()
      syn_arr[mask_prim] = mean_color

      base, _ = os.path.splitext(name)
      syn_name = f"{base}_synthetic.jpg"
      Image.fromarray(syn_arr).save(os.path.join(output_folder, syn_name))

      info.append({"filename": syn_name, "predominant_class": sec_label, "area": sec_area})

  pd.DataFrame(info).to_csv(os.path.join(output_folder, "new_dataset.csv"), index=False)
  print(f"Procesamiento completado en {os.path.basename(input_folder)}. CSV guardado.")

# Lectura de anotaciones para conjuntos de entrenamiento y validación
train_ann = pd.read_csv(os.path.join(train_dir, "annotations.csv"))
valid_ann = pd.read_csv(os.path.join(valid_dir, "annotations.csv"))

process_folder(train_dir, processed_train_dir, train_ann)
process_folder(valid_dir, processed_valid_dir, valid_ann)

print("Todos los conjuntos han sido procesados correctamente.")

Procesando train:  64%|██████▍   | 288/448 [00:05<00:02, 64.35it/s]

No se encontraron anotaciones para la imagen: IMG_3133_jpeg_jpg.rf.f439b9d382fd153b96f0a88cdf169172.jpg


Procesando train: 100%|██████████| 448/448 [00:07<00:00, 57.55it/s]


Procesamiento completado en train. CSV guardado.


Procesando valid: 100%|██████████| 127/127 [00:01<00:00, 63.54it/s]

Procesamiento completado en valid. CSV guardado.
Todos los conjuntos han sido procesados correctamente.
